In [2]:
print("all_ok")

all_ok


In [5]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [7]:
model= ChatGroq(model = "qwen/qwen3-32b")

In [8]:
model.invoke("where is new delhi?")

AIMessage(content='<think>\nOkay, the user is asking "where is New Delhi?" I need to provide a clear and concise answer. Let me start by recalling that New Delhi is the capital of India. I should mention its location within India, maybe specify the state it\'s in, which is Delhi. Wait, but Delhi is actually a union territory, right? So I should clarify that. \n\nNext, for someone who might not know much about India\'s geography, it might help to mention neighboring countries or major cities. For example, it\'s near the border with Pakistan, but not too close. Maybe mention states around it, like Haryana and Uttar Pradesh. Also, its position in northern India.\n\nI should include some geographical coordinates to be precise. Let me check the approximate latitude and longitude. I think it\'s around 28.6139° N, 77.2090° E. That\'s correct. Also, the altitude is about 217 meters above sea level.\n\nWhat else? New Delhi is part of the National Capital Territory of Delhi. Maybe explain that i